In [ ]:
import tensorflow as tf
from tensorflow.keras import backend

In [ ]:
tf.__version__

In [ ]:
!pip install imageio

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import imageio
import matplotlib.pyplot as plt
import os, time
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
batch_size = 10
data_generator = ImageDataGenerator(rescale=1./255)

train_generator = data_generator.flow_from_directory(
    '/content/drive/My Drive/Virat/cropped_train',
    target_size = (32,32),
    batch_size = batch_size)

In [ ]:
train_images, train_labels = train_generator.next()

In [ ]:
train_images.shape

In [ ]:
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

In [ ]:
img_shape = (32, 32, 3)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Dropout, BatchNormalization, LeakyReLU, Reshape, Conv2DTranspose

In [ ]:
def build_generator():
    model = Sequential()
    model.add(Dense(8 * 8 * 256, activation = 'relu', input_dim=100))
    model.add(Reshape((8, 8, 256)))
    model.add(BatchNormalization())
    model.add(Conv2DTranspose(128, kernel_size = 4, strides = 2, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization())
    model.add(Conv2DTranspose(64, kernel_size = 4, strides = 2, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization())
    model.add(Conv2DTranspose(3, kernel_size = 4, strides = 1, padding='same', activation = 'tanh'))
    return model
def build_discriminator():
    model = Sequential()
    model.add(Conv2D(64, kernel_size = 4, strides = 2, padding='same', input_shape = img_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Conv2D(128, kernel_size = 4, strides = 2, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(1, activation = 'sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))
    return model


In [ ]:
from tensorflow.keras.optimizers import Adam


generator = build_generator()

z = tf.keras.Input(shape=(100,))
img = generator(z)

discriminator = build_discriminator()
discriminator.trainable = False
valid_pred = discriminator(img)

combined = tf.keras.Model(z, valid_pred)
optimizer = Adam(0.0002, 0.5)
combined.compile(loss = 'binary_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
combined.summary()



In [ ]:
def train(epochs, batch_size, save_interval = 30):
    X_train = next(train_generator)[0]
    x_train = (X_train - 0.5) * 2
    min_batch = int(batch_size / 2)

    for epoch in range(epochs):
        id = np.random.randint(0, X_train.shape[0], min_batch)
        x_real = x_train[id]

        noise = np.random.normal(0, 1, (min_batch, 100))
        x_fake = generator.predict(noise)

        d_loss_real = discriminator.train_on_batch(x_real, np.ones((min_batch, 1)))
        d_loss_fake = discriminator.train_on_batch(x_fake, np.zeros((min_batch, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        if isinstance(d_loss_real, (list, tuple)):
            d_loss_real_val = d_loss_real[0]
            d_acc_real = d_loss_real[1]
        else:
            d_loss_real_val = d_loss_real
            d_acc_real = 0

        if isinstance(d_loss_fake, (list, tuple)):
            d_loss_fake_val = d_loss_fake[0]
            d_acc_fake = d_loss_fake[1]
        else:
            d_loss_fake_val = d_loss_fake
            d_acc_fake = 0

        d_loss = 0.5 * (d_loss_real_val + d_loss_fake_val)
        d_acc = 0.5 * (d_acc_real + d_acc_fake)


        noise = np.random.normal(0, 1, (min_batch, 100))
        valid_y = np.ones((min_batch, 1))
        g_loss = combined.train_on_batch(noise, valid_y)

        if isinstance(g_loss, (list, tuple)):
            g_loss_val = g_loss[0]
        else:
            g_loss_val = g_loss
        print(f"{epoch} [D loss: {d_loss}, acc.: {100*d_acc}] [G loss: {g_loss_val}]")

        if epoch % save_interval == 0:
            print(f"Saving images at epoch {epoch}")
            save_imgs(epoch)

def save_imgs(epoch):
    noise = np.random.normal(0, 1, (16, 100))
    gen_imgs = generator.predict(noise)
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(4, 4)
    count = 0
    for i in range(4):
        for j in range(4):
            axs[i,j].imshow(gen_imgs[count])
            axs[i,j].axis('off')
            count += 1
    save_dir = "/content/drive/My Drive/Virat/Output"
    os.makedirs(save_dir, exist_ok=True)

    fig.savefig(os.path.join(save_dir, 'image_at_epoch_{:04d}.png'.format(epoch)))
    plt.close()
    print(f"Image saved for epoch {epoch}")


In [ ]:
train(epochs=5001, batch_size=32, save_interval = 500)

In [ ]:

import numpy as np
from PIL import Image

image_files = sorted([f for f in os.listdir('/content/drive/My Drive/Virat/Output') if f.endswith('.png')])


with imageio.get_writer('/content/drive/My Drive/Virat/Output/gan.gif', mode='I') as writer:
    for filename in image_files:
        image = Image.open(os.path.join('/content/drive/My Drive/Virat/Output', filename))
        writer.append_data(np.array(image))

from IPython.display import Image
Image(filename='/content/drive/My Drive/Virat/Output/gan.gif')
